In [14]:
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

from tqdm import tqdm
from itertools import product
import pandas as pd
import pandas_datareader.data as web
import numpy as np
from numpy.linalg import LinAlgError

import statsmodels.tsa.api as tsa
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, q_stat, adfuller
from scipy.stats import probplot, moment
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

In [15]:
def plot_correlogram(x, lags=None, title=None):
    lags = min(10, int(len(x) / 5)) if lags is None else lags
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14, 8))
    x.plot(ax=axes[0][0], title='Residuals')
    x.rolling(21).mean().plot(ax=axes[0][0], c='k', lw=1)
    q_p = np.max(q_stat(acf(x, nlags=lags), len(x))[1])
    stats = f'Q-Stat: {np.max(q_p):>8.2f}\nADF: {adfuller(x)[1]:>11.2f}'
    axes[0][0].text(x=.02, y=.85, s=stats, transform=axes[0][0].transAxes)
    probplot(x, plot=axes[0][1])
    mean, var, skew, kurtosis = moment(x, moment=[1, 2, 3, 4])
    s = f'Mean: {mean:>12.2f}\nSD: {np.sqrt(var):>16.2f}\nSkew: {skew:12.2f}\nKurtosis:{kurtosis:9.2f}'
    axes[0][1].text(x=.02, y=.75, s=s, transform=axes[0][1].transAxes)
    plot_acf(x=x, lags=lags, zero=False, ax=axes[1][0])
    plot_pacf(x, lags=lags, zero=False, ax=axes[1][1])
    axes[1][0].set_xlabel('Lag')
    axes[1][1].set_xlabel('Lag')
    fig.suptitle(title, fontsize=14)
    fig.subplots_adjust(top=.9)

In [16]:
# chk: https://fred.stlouisfed.org/series/IPGMFN
# IPGMFN monthly data
industrial_production = web.DataReader('IPGMFN', 'fred', '2000', '2022-12-31').squeeze().dropna()
# NASDAQCOM daily data
# chk: https://www.nasdaq.com/market-activity/index/comp
nasdaq = web.DataReader('NASDAQCOM', 'fred', '2000', '2022-12-31').squeeze().dropna()

In [17]:
# log transformation
nasdaq_log = np.log(nasdaq)
industrial_production_log = np.log(industrial_production)

# Differencing
nasdaq_log_diff = nasdaq_log.diff().dropna()
# seasonal differencing => yoy instantanteous returns
industrial_production_log_diff = industrial_production_log.diff(12).dropna()

In [18]:
industrial_production_log_diff.tail()

DATE
2022-08-01    0.025688
2022-09-01    0.040818
2022-10-01    0.025143
2022-11-01    0.009435
2022-12-01   -0.009811
Name: IPGMFN, dtype: float64

In [19]:
# ARMA vs ARIMA
model1 = tsa.ARIMA(endog=nasdaq_log_diff, order=(2, 0, 2)).fit()
model2 = tsa.ARIMA(endog=nasdaq_log, order=(2, 1, 2)).fit()

In [ ]:
model1.params.sort_index() == model2.params.sort_index().values

In [ ]:
model1 = tsa.ARMA(endog=nasdaq_log_diff, order=(2, 2)).fit()
model2 = tsa.ARIMA(endog=nasdaq_log, order=(2, 1, 2)).fit()

In [ ]:
model1.params.sort_index() == model2.params.sort_index().values

In [ ]:
# Seasonal differencing vs SARIMAX
model1 = tsa.statespace.SARIMAX(industrial_production_log, order=(2, 0, 2), seasonal_order=(0, 1, 0, 12)).fit()
model2 = tsa.statespace.SARIMAX(industrial_production_log_diff, order=(2, 0, 2), seasonal_order=(0, 0, 0, 12)).fit()

In [ ]:
model1.params.to_frame('SARIMAX').join(model2.params.to_frame('diff'))

In [ ]:
train_size = 120
results = {}
y_true = industrial_production_log_diff.iloc[train_size:]
for p in range(5):
    for q in range(5):
        aic, bic = [], []
        if p == 0 and q == 0:
            continue
        print(p, q)
        convergence_error = stationarity_error = 0
        y_pred = []
        for T in range(train_size, len(industrial_production_log_diff)):
            train_set = industrial_production_log_diff.iloc[T - train_size:T]
            try:
                model = tsa.ARMA(endog=train_set, order=(p, q)).fit()
            except LinAlgError:
                convergence_error += 1
            except ValueError:
                stationarity_error += 1

            forecast, _, _ = model.forecast(steps=1)
            y_pred.append(forecast[0])
            aic.append(model.aic)
            bic.append(model.bic)

        result = (pd.DataFrame({'y_true': y_true, 'y_pred': y_pred}).replace(np.inf, np.nan).dropna())

        rmse = np.sqrt(mean_squared_error(y_true=result.y_true, y_pred=result.y_pred))

        results[(p, q)] = [rmse, np.mean(aic), np.mean(bic), convergence_error, stationarity_error]

In [ ]:
arma_results = pd.DataFrame(results).T
arma_results.columns = ['RMSE', 'AIC', 'BIC', 'convergence', 'stationarity']
arma_results.index.names = ['p', 'q']
arma_results.info()

In [ ]:
with pd.HDFStore('data/arima.h5') as store:
    store.put('arma', arma_results)

In [ ]:
arma_results.nsmallest(5, columns=['RMSE'])
arma_results.nsmallest(5, columns=['BIC'])
fig, axes = plt.subplots(ncols=2, figsize=(10, 4), sharex=True, sharey=True)
sns.heatmap(arma_results[arma_results.RMSE < .5].RMSE.unstack().mul(10), fmt='.3f', annot=True, cmap='Blues',
            ax=axes[0], cbar=False);
sns.heatmap(arma_results.BIC.unstack(), fmt='.2f', annot=True, cmap='Blues', ax=axes[1], cbar=False)
axes[0].set_title('Root Mean Squared Error')
axes[1].set_title('Bayesian Information Criterion')
arma_results.rank().loc[:, ['RMSE', 'BIC']].mean(1).nsmallest(5)

In [ ]:
best_p, best_q = arma_results.rank().loc[:, ['RMSE', 'BIC']].mean(1).idxmin()
best_arma_model = tsa.ARMA(endog=industrial_production_log_diff, order=(best_p, best_q)).fit()
print(best_arma_model.summary())

In [ ]:
plot_correlogram(best_arma_model.resid)

In [ ]:
sarimax_model = tsa.SARIMAX(endog=industrial_production_log_diff.dropna().values, order=(2, 0, 2),
                            seasonal_order=(1, 0, 1, 12)).fit(start_params=[0, 0, 0, 0, 0, 0, 1])
print(sarimax_model.summary())
plot_correlogram(pd.Series(sarimax_model.resid))

In [ ]:
l3 = list(range(3))
l4 = list(range(4))
params = [t for t in product(l4, l4, l3, l3) if t[0] > 0 and t[1] > 0]
len(params)
train_size = 120  # 10 years of training data
results = {}
test_set = industrial_production_log_diff.iloc[train_size:]

for p1, q1, p2, q2 in tqdm(params):
    preds = test_set.copy().to_frame('y_true').assign(y_pred=np.nan)
    aic, bic = [], []
    if p1 == 0 and q1 == 0:
        continue
    convergence_error = stationarity_error = 0
    y_pred = []
    for i, T in enumerate(range(train_size, len(industrial_production_log_diff))):
        train_set = industrial_production_log_diff.iloc[T - train_size:T]
        try:
            with warnings.catch_warnings():
                warnings.filterwarnings("ignore")
                model = tsa.SARIMAX(endog=train_set.values,
                                    order=(p1, 0, q1),
                                    seasonal_order=(p2, 0, q2, 12)).fit(disp=0)
        except LinAlgError:
            convergence_error += 1
        except ValueError:
            stationarity_error += 1

        preds.iloc[i, 1] = model.forecast(steps=1)[0]
        aic.append(model.aic)
        bic.append(model.bic)

    preds.dropna(inplace=True)
    mse = mean_squared_error(preds.y_true, preds.y_pred)
    results[(p1, q1, p2, q2)] = [np.sqrt(mse), preds.y_true.sub(preds.y_pred).pow(2).std(), np.mean(aic),
                                 np.std(aic), np.mean(bic), np.std(bic), convergence_error, stationarity_error]

In [ ]:
sarimax_results = pd.DataFrame(results).T
sarimax_results.columns = ['RMSE', 'RMSE_std', 'AIC', 'AIC_std', 'BIC', 'BIC_std', 'convergence', 'stationarity']
sarimax_results['CV'] = sarimax_results.RMSE_std.div(sarimax_results.RMSE)
sarimax_results.index.names = ['p1', 'q1', 'p2', 'q2']
sarimax_results.info()
with pd.HDFStore('data/arima.h5') as store:
    store.put('sarimax', sarimax_results)
with pd.HDFStore('data/arima.h5') as store:
    sarimax_results = store.get('sarimax')
sarimax_results.nsmallest(5, columns='RMSE')

In [ ]:
sarimax_results[['RMSE', 'AIC', 'BIC']].sort_values('RMSE').head()
sarimax_results[['RMSE', 'AIC', 'BIC']].corr('spearman')
sns.jointplot(y='RMSE', x='BIC', data=sarimax_results[['RMSE', 'BIC']].rank());
sarimax_results[(sarimax_results.RMSE < sarimax_results.RMSE.quantile(.05)) &
                (sarimax_results.BIC < sarimax_results.BIC.quantile(.1))].sort_values('RMSE')

In [ ]:
p1, q1, p2, q2 = 2, 3, 1, 2
best_model = tsa.SARIMAX(endog=industrial_production_log_diff.values, order=(p1, 0, q1),
                         seasonal_order=(p2, 0, q2, 12)).fit()
print(best_model.summary())

In [ ]:
plot_correlogram(pd.Series(best_model.resid), lags=20,
                 title=f'SARIMAX ({p1}, 0, {q1}) x ({p2}, 0, {q2}, 12) | Model Diagnostics')